In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pylab as pl
import seaborn as sns

In [ ]:
df = pd.read_csv("/mnt/data/ML-Ready/mixed_data/outputs/best_val_loss_res.csv")

In [ ]:
df2 = pd.read_csv("/mnt/data/ML-Ready/mixed_data/outputs/deep-vit.csv")

In [ ]:
df3 = pd.read_csv("/mnt/data/ML-Ready/flares_event_dir/outputs/testing.csv")

In [ ]:
print(df.head())

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%dT%H:%M:%S')
df2['Timestamp'] = pd.to_datetime(df2['Timestamp'], format='%Y-%m-%dT%H:%M:%S')
# df3['Timestamp'] = pd.to_datetime(df3['Timestamp'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')

In [ ]:
plt.style.use('dark_background')
sns.set_palette("husl")

df = df.sort_values(by='Timestamp')

# Plot
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#2d2d44')
plt.plot(df['Timestamp'], df['Predictions'], label='Predicted SXR - MEGS', marker='o',markersize=4,alpha=.75,c='skyblue')
plt.plot(df2['Timestamp'], df2['Predictions'], label='Predicted SXR - Deep-ViT', marker='o',markersize=4,alpha=.75,c='orange')
plt.plot(df['Timestamp'], df['ground_truth'], label='Ground Truth SXR', marker='x',markersize=4,alpha=.75,c='red')
plt.xlabel("Timestamp")
plt.ylabel("SXR Value (W/m²)")
plt.title("SXR Prediction vs Ground Truth")
plt.xlim(pd.to_datetime("2023-08-07T00:00:00"), pd.to_datetime("2023-08-9T00:00:00"))

plt.yscale('log')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.xticks(rotation=0)
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Sort dataframes by timestamp
df = df.sort_values(by='Timestamp')
df2 = df2.sort_values(by='Timestamp')

# Create the interactive plot
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(
    x=df['Timestamp'],
    y=df['Predictions'],
    mode='lines+markers',
    name='Predicted SXR - MEGS',
    line=dict(color='skyblue'),
    marker=dict(size=4, opacity=0.75),
    hovertemplate='<b>MEGS Prediction</b><br>Time: %{x}<br>SXR: %{y:.2e} W/m²<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=df2['Timestamp'],
    y=df2['Predictions'],
    mode='lines+markers',
    name='Predicted SXR - Deep-ViT',
    line=dict(color='orange'),
    marker=dict(size=4, opacity=0.75),
    hovertemplate='<b>Deep-ViT Prediction</b><br>Time: %{x}<br>SXR: %{y:.2e} W/m²<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=df['Timestamp'],
    y=df['ground_truth'],
    mode='lines+markers',
    name='Ground Truth SXR',
    line=dict(color='red'),
    marker=dict(symbol='x', size=4, opacity=0.75),
    hovertemplate='<b>Ground Truth</b><br>Time: %{x}<br>SXR: %{y:.2e} W/m²<extra></extra>'
))

# Update layout with dark theme
fig.update_layout(
    width=1600,  # Width in pixels
    height=800,
    title={
        'text': 'SXR Prediction vs Ground Truth (Interactive)',
        'x': 0.5,
        'font': {'size': 16, 'color': 'white'}
    },
    xaxis_title="Timestamp",
    yaxis_title="SXR Value (W/m²)",
    xaxis=dict(
        range=[pd.to_datetime("2023-08-07T00:00:00"), pd.to_datetime("2023-08-09T00:00:00")],
        color='white',
        gridcolor='rgba(255,255,255,0.2)',
        showgrid=True
    ),
    yaxis=dict(
        type='log',
        color='white',
        gridcolor='rgba(255,255,255,0.2)',
        showgrid=True
    ),
    plot_bgcolor='#2d2d44',
    paper_bgcolor='#1a1a2e',
    font=dict(color='white'),
    legend=dict(
        bgcolor='rgba(0,0,0,0.5)',
        bordercolor='white',
        borderwidth=1
    )
)

# Enable range slider and range selector for easy time navigation
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=6, label="6h", step="hour", stepmode="backward"),
                dict(count=12, label="12h", step="hour", stepmode="backward"),
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(step="all")
            ])
        ),
    )
)

# Show the plot
fig.show()

In [ ]:
from matplotlib.colors import LogNorm, AsinhNorm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from pathlib import Path
import glob
from IPython.display import display, clear_output
import ipywidgets as widgets
import sunpy.visualization.colormaps.cm  # Register colormaps


def load_aia_image(timestamp, aia_dir):
    """Load AIA image for given timestamp"""
    # Adjust this pattern based on your AIA file naming convention
    pattern = f"{aia_dir}/*{timestamp}*"
    files = glob.glob(pattern)
    print(f"Found {len(files)} files matching pattern: {pattern}")
    print(files)
    if files:
        # Load your AIA image here - adjust based on your format (.fits, .npy, etc.)
        # For example, if it's a numpy array:
        aia_data = np.load(files[0])
        # Or if it's a FITS file:
        # from astropy.io import fits
        # aia_data = fits.getdata(files[0])

        # For now, return a placeholder - replace with your actual loading code
        #aia_data = np.random.random((512, 512))  # Replace this line
        return aia_data
    return None

def load_attention_map(timestamp, weight_path):
    """Load attention map for given timestamp"""
    filepath = f"{weight_path}{timestamp}"
    try:
        attention_map = np.loadtxt(filepath, delimiter=",")
        return attention_map
    except:
        return None

def display_images_for_timestamp(timestamp, aia_dir, weight_path):
    """Load and display AIA image and attention map for a specific timestamp"""
    print(f"\nLoading images for timestamp: {timestamp}")

    # Load AIA image and attention map
    aia_data = load_aia_image(timestamp, aia_dir)
    attention_data = load_attention_map(timestamp, weight_path)

    if aia_data is None:
        print(f"Could not load AIA image for {timestamp}")
        return

    if attention_data is None:
        print(f"Could not load attention map for {timestamp}")
        return

    # Create the visualization
    fig, ax = plt.subplots(6, 2, figsize=(5, 10))
    fig.patch.set_facecolor('#1a1a2e')

    # AIA image
    wav = ['94', '131', '171', '193', '211', '304']
    for i, w in enumerate(wav):
        im1 = ax[i][0].imshow(aia_data[i], cmap='sdoaia'+w, origin='lower')
        #ax[i][0].set_title(f'AIA Image - {timestamp} - {w}', fontsize=12, color='white')
        ax[i][0].axis('off')
        im2 = ax[i][1].imshow(attention_data, cmap='viridis', origin='lower', alpha=0.8,
                     norm=AsinhNorm())
        #ax[i][1].set_title(f'Attention Map (Log Scale) - {timestamp}', fontsize=12, color='white')
        ax[i][1].axis('off')

    # Add colorbar for AIA
    # cbar1 = plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
    # cbar1.ax.tick_params(colors='white', labelsize=10)

    # Attention map


    # # Add colorbar for attention
    # cbar2 = plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
    # cbar2.ax.tick_params(colors='white', labelsize=10)
    # cbar2.set_label('Attention Weight', color='white', fontsize=10)

    plt.tight_layout()
    plt.show()

class InteractiveSXRPlot:
    """Interactive plot class that handles click events"""

    def __init__(self, df, df2, aia_dir, weight_path):
        self.df = df.sort_values(by='Timestamp').reset_index(drop=True)
        self.df2 = df2.sort_values(by='Timestamp').reset_index(drop=True)
        self.aia_dir = aia_dir
        self.weight_path = weight_path
        self.fig = None
        self.create_plot()

    def create_plot(self):
        """Create the main interactive plot"""
        self.fig = go.Figure()

        # Add MEGS predictions
        self.fig.add_trace(go.Scatter(
            x=self.df['Timestamp'],
            y=self.df['Predictions'],
            mode='lines+markers',
            name='Predicted SXR - MEGS',
            line=dict(color='skyblue'),
            marker=dict(size=6, opacity=0.75),
            hovertemplate='<b>MEGS Prediction</b><br>Time: %{x}<br>SXR: %{y:.2e} W/m²<br><i>Click to see AIA + Attention</i><extra></extra>',
            customdata=self.df['Timestamp']  # Store timestamps for click handling
        ))

        # Add Deep-ViT predictions
        self.fig.add_trace(go.Scatter(
            x=self.df2['Timestamp'],
            y=self.df2['Predictions'],
            mode='lines+markers',
            name='Predicted SXR - Deep-ViT',
            line=dict(color='orange'),
            marker=dict(size=6, opacity=0.75),
            hovertemplate='<b>Deep-ViT Prediction</b><br>Time: %{x}<br>SXR: %{y:.2e} W/m²<extra></extra>'
        ))

        # Add ground truth
        self.fig.add_trace(go.Scatter(
            x=self.df['Timestamp'],
            y=self.df['ground_truth'],
            mode='lines+markers',
            name='Ground Truth SXR',
            line=dict(color='red'),
            marker=dict(symbol='x', size=6, opacity=0.75),
            hovertemplate='<b>Ground Truth</b><br>Time: %{x}<br>SXR: %{y:.2e} W/m²<extra></extra>'
        ))

        # Update layout
        self.fig.update_layout(
            width=1400,
            height=800,
            title={
                'text': 'Interactive SXR Plot - Click points to see AIA Images & Attention Maps',
                'x': 0.5,
                'font': {'size': 16, 'color': 'white'}
            },
            xaxis_title="Timestamp",
            yaxis_title="SXR Value (W/m²)",
            xaxis=dict(
                range=[pd.to_datetime("2023-08-07T00:00:00"), pd.to_datetime("2023-08-09T00:00:00")],
                color='white',
                gridcolor='rgba(255,255,255,0.2)',
                showgrid=True
            ),
            yaxis=dict(
                type='log',
                color='white',
                gridcolor='rgba(255,255,255,0.2)',
                showgrid=True
            ),
            plot_bgcolor='#2d2d44',
            paper_bgcolor='#1a1a2e',
            font=dict(color='white'),
            legend=dict(
                bgcolor='rgba(0,0,0,0.5)',
                bordercolor='white',
                borderwidth=1
            ),
            hovermode='closest'
        )

        # Add range slider and selector
        self.fig.update_layout(
            xaxis=dict(
                rangeslider=dict(visible=True),
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1h", step="hour", stepmode="backward"),
                        dict(count=6, label="6h", step="hour", stepmode="backward"),
                        dict(count=12, label="12h", step="hour", stepmode="backward"),
                        dict(count=1, label="1d", step="day", stepmode="backward"),
                        dict(step="all")
                    ])
                ),
            )
        )

    def on_click(self, trace, points, selector):
        """Handle click events on the plot"""
        if points.point_inds:
            # Get the clicked point index
            point_idx = points.point_inds[0]

            # Get timestamp from the clicked point
            if trace.name == 'Predicted SXR - MEGS':
                timestamp = self.df.iloc[point_idx]['Timestamp']
            else:
                # For other traces, find closest timestamp in df
                clicked_time = pd.to_datetime(trace.x[point_idx])
                closest_idx = (pd.to_datetime(self.df['Timestamp']) - clicked_time).abs().idxmin()
                timestamp = self.df.iloc[closest_idx]['Timestamp']

            # Display images for this timestamp
            display_images_for_timestamp(timestamp, self.aia_dir, self.weight_path)

    def show(self):
        """Display the interactive plot"""
        # Note: Click handling works best in Jupyter notebooks
        # For standalone scripts, you might need to use a different approach
        self.fig.show()
        print("\nInstructions:")
        print("1. Click on any point in the MEGS prediction line (blue) to see AIA image and attention map")
        print("2. Use the range slider and buttons to navigate through time")
        print("3. Zoom and pan to explore specific time periods")
        return self.fig

def create_simple_interactive_plot(df, df2, aia_dir, weight_path):
    """
    Simple function to create interactive plot with manual timestamp input
    Use this if click handling doesn't work in your environment
    """

    # Create the plot
    plot_instance = InteractiveSXRPlot(df, df2, aia_dir, weight_path)
    fig = plot_instance.show()

    # Create a simple interface for manual timestamp input
    def show_images_for_timestamp(timestamp_str):
        """Helper function to display images for a manually entered timestamp"""
        display_images_for_timestamp(timestamp_str, aia_dir, weight_path)

    print(f"\nAvailable timestamps range from {df['Timestamp'].min()} to {df['Timestamp'].max()}")
    print("\nTo view images for a specific timestamp, call:")
    print("show_images_for_timestamp('your_timestamp_here')")

    return fig, show_images_for_timestamp

# Usage example:
"""
# Method 1: Automatic click handling (works in Jupyter)
plot = InteractiveSXRPlot(df, df2, "/path/to/aia/images", "/path/to/weights/")
fig = plot.show()

# Method 2: Manual timestamp input (works anywhere)
fig, show_images = create_simple_interactive_plot(df, df2, "/path/to/aia/images", "/path/to/weights/")
# Then call: show_images('2023-08-07T12:34:56')
"""

print("Interactive plot created!")
print("Use: plot = InteractiveSXRPlot(df, df2, aia_dir, weight_path)")
print("Then: fig = plot.show()")
print("Click on blue points to see AIA images and attention maps!")

In [ ]:
dat = pd.read_csv("/mnt/data/ML-Ready/mixed_data/weights/2023-08-01T00:00:00")

In [ ]:
plt.imshow(dat)

In [ ]:
fig, show_images = create_simple_interactive_plot(df, df2, "/mnt/data/ML-Ready/mixed_data/AIA/test/", "/mnt/data/ML-Ready/mixed_data/weights/")
show_images('2023-08-03T16:37:00')

In [ ]:
fig = InteractiveSXRPlot(df, df2, "/mnt/data/ML-Ready/mixed_data/AIA/test/", "/mnt/data/ML-Ready/mixed_data/weights/")
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.animation import FuncAnimation
from datetime import datetime, timedelta

# Set style
plt.style.use('dark_background')
sns.set_palette("husl")

# Ensure df is sorted by timestamp
df = df.sort_values(by='Timestamp')

# Convert timestamps to datetime if they aren't already
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Define date range for the animation
start_date = pd.to_datetime("2023-08-01T00:00:00")
end_date = pd.to_datetime("2023-08-13T23:59:59")


# Define step size for smooth animation
step_hours = .2 # Move window by 0.05 hours (3 minutes) each frame

# Calculate number of frames correctly
# We want the window to slide from start_date to (end_date - 1 day)
# so the last frame shows end_date-1day to end_date
animation_duration_hours = (end_date - start_date - timedelta(days=1)).total_seconds() / 3600
n_frames = int(animation_duration_hours / step_hours) + 1


# Create figure and axis
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#2d2d44')

# Initialize empty line objects
line_pred, = ax.plot([], [], label='Predicted SXR', marker='o', markersize=4, alpha=0.75, c='skyblue')
line_truth, = ax.plot([], [], label='Ground Truth SXR', marker='x', markersize=4, alpha=0.75, c='orange')

# Set up the plot
ax.set_xlabel("Timestamp", fontsize=12)
ax.set_ylabel("SXR Value (W/m²)", fontsize=12)
ax.set_title("SXR Prediction vs Ground Truth - Rolling 1-Day Window", fontsize=14)
ax.set_yscale('log')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

def animate(frame):
    # Calculate the current window start time (moving by step_hours each frame)
    window_start = start_date + timedelta(hours=frame * step_hours)
    window_end = window_start + timedelta(days=1)  # Always 24-hour window

    # Filter data for current window
    mask = (df['Timestamp'] >= window_start) & (df['Timestamp'] < window_end)
    window_df = df[mask]

    if len(window_df) > 0:
        # Update line data
        line_pred.set_data(window_df['Timestamp'], window_df['Predictions'])
        line_truth.set_data(window_df['Timestamp'], window_df['ground_truth'])

        # Set axis limits
        ax.set_xlim(window_start, window_end)

        # Dynamic y-axis limits based on current window data
        all_values = pd.concat([window_df['Predictions'], window_df['ground_truth']])
        if len(all_values) > 0:
            y_min = all_values.min() * 0.5
            y_max = all_values.max() * 2
            ax.set_ylim(y_min, y_max)

    # Update title with current date and time
    ax.set_title(f"SXR Prediction vs Ground Truth - {window_start.strftime('%Y-%m-%d %H:%M')} to {window_end.strftime('%Y-%m-%d %H:%M')}", fontsize=12)

    # Format x-axis ticks
    ax.tick_params(axis='x', rotation=0)

    return line_pred, line_truth

# Create animation
# Now we have many more frames for smooth transition
anim = FuncAnimation(fig, animate, frames=n_frames, interval=50, blit=False, repeat=True)

plt.tight_layout()

# Save as MP4 (requires ffmpeg) - higher fps for smooth playback
# anim.save('sxr_rolling_window.mp4', writer='ffmpeg', fps=20, bitrate=1800)

# Save as GIF (alternative) - higher fps for smooth playback
anim.save('sxr_rolling_window_baseline_mixed.gif', writer='pillow', fps=10)

plt.show()

# To save the animation, uncomment one of the save lines above
# Make sure you have ffmpeg installed for MP4 or use the GIF option
# Note: The animation will be much longer now due to the smooth transitions

In [ ]:
df = df.sort_values(by='Timestamp')

# Plot
plt.figure(figsize=(10, 5))
#plt.plot(df['Timestamp'], df['Predictions'], label='Predicted SXR', marker='o',markersize=2,alpha=.5)
plt.plot(df['Timestamp'], df['ground_truth'], label='Ground Truth SXR', marker='x',markersize=2,alpha=.5)
plt.plot(df3['Timestamp'], df3['ground_truth'], label='Predicted SXR', marker='o',markersize=2,alpha=.5)
#plt.plot(df2['Timestamp'], df2['Predictions'], label='Predicted SXR Best Checkpoint', marker='o',markersize=2,alpha=.5)
plt.xlabel("Timestamp")
plt.ylabel("SXR Value (W/m²)")
plt.title("SXR Prediction vs Ground Truth")
plt.xlim(pd.to_datetime("2023-08-01T00:00:00"), pd.to_datetime("2023-08-1T06:00:00"))
plt.yscale('log')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.style.use('dark_background')
df = df.sort_values(by='Timestamp')

# Filter out NaN, None, infinite values, and values <= 0 (for log scale)
df_clean = df[df['ground_truth'].notna() & np.isfinite(df['ground_truth']) & (df['ground_truth'] > 0)]
df3_clean = df3[df3['ground_truth'].notna() & np.isfinite(df3['ground_truth']) & (df3['ground_truth'] > 0)]

# Plot
plt.figure(figsize=(7, 5))
#plt.plot(df['Timestamp'], df['Predictions'], label='Predicted SXR', marker='o',markersize=2,alpha=.5)
# Option 1: Use scatter plot instead of line plot to avoid connecting points
plt.scatter(df_clean['Timestamp'], df_clean['ground_truth'], label='Non-flaring Data', marker='x', s=25, alpha=1, c='skyblue')
plt.scatter(df3_clean['Timestamp'], df3_clean['ground_truth'], label='Flaring Data', marker='o', s=25, alpha=1, c='orange')


plt.xlabel("Timestamp")
plt.ylabel("SXR Value (W/m²)")
#plt.title("SXR Prediction vs Ground Truth")
plt.xlim(pd.to_datetime("2023-08-01T04:00:00"), pd.to_datetime("2023-08-1T06:00:00"))
plt.yscale('log')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()

In [ ]:

import matplotlib.pyplot as plt
import numpy as np


# Enhanced version with better log scale handling

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#2d2d44')

# Create log-spaced bins for better visualization with log scale
log_bins = np.logspace(np.log10(df['ground_truth'].min()),
                       np.log10(df['ground_truth'].max()), 50)

plt.hist2d(df['ground_truth'], df['Predictions'],
           bins=[log_bins, log_bins], cmap='summer', norm='log')
plt.colorbar(label='Count (log scale)')

# Add equality line
min_val = min(df['ground_truth'].min(), df['Predictions'].min())
max_val = max(df['ground_truth'].max(), df['Predictions'].max())
plt.plot([min_val, max_val], [min_val, max_val],
         label='Perfect Prediction', color='red', linestyle='--', linewidth=2,)

plt.xlabel("Ground Truth SXR")
plt.ylabel("Predicted SXR")
plt.title("SXR Prediction vs Ground Truth (Log-binned 2D Histogram)")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid(True, alpha=0.3, color='white')
plt.tight_layout()
#plt.savefig('sxr_prediction_vs_ground_truth_histogram_mixed.png', dpi=500, bbox_inches='tight',)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Enhanced version with better log scale handling

fig, ax = plt.subplots(1, 2, figsize=(12, 8))
#fig.patch.set_facecolor('#1a1a2e')
#ax.set_facecolor('#2d2d44')

# Create log-spaced bins for better visualization with log scale
log_bins = np.logspace(np.log10(df2['ground_truth'].min()),
                       np.log10(df2['ground_truth'].max()), 50)

log_bins = np.logspace(np.log10(df['ground_truth'].min()),
                       np.log10(df['ground_truth'].max()), 50)

ax[0].hist2d(df['ground_truth'], df['Predictions'],
           bins=[log_bins, log_bins], cmap='summer', norm='log')

ax[1].hist2d(df2['ground_truth'], df2['Predictions'],
           bins=[log_bins, log_bins], cmap='summer', norm='log')

#plt.colorbar(label='Count (log scale)')

# Add equality line
min_val = min(df2['ground_truth'].min(), df2['Predictions'].min())
max_val = max(df2['ground_truth'].max(), df2['Predictions'].max())
ax[0].plot([min_val, max_val], [min_val, max_val],
         label='Perfect Prediction', color='red', linestyle='--', linewidth=2,)
ax[1].plot([min_val, max_val], [min_val, max_val],
         label='Perfect Prediction', color='red', linestyle='--', linewidth=2,)

plt.xlabel("Ground Truth SXR")
plt.ylabel("Predicted SXR")
plt.title("SXR Prediction vs Ground Truth (Log-binned 2D Histogram)")
ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[1].set_xscale("log")
ax[1].set_yscale("log")
#plt.xscale("log")
#plt.yscale("log")
plt.legend()
plt.grid(True, alpha=0.3, color='white')
plt.tight_layout()
plt.savefig('sxr_prediction_vs_ground_truth_histogram_mixed.png', dpi=500, bbox_inches='tight',)
plt.show()

In [ ]:
df3 = pd.read_csv("/mnt/data/goes_combined/combined_g18_avg1m_20230701_20230815.csv")

In [ ]:
print(df3.head())

In [ ]:
df3['time'] = pd.to_datetime(df3['time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [ ]:
print(df3.head())

In [ ]:
df3 =df3[df3['quad_diode']==0]

In [ ]:
print(df3['xrsb_flux'])

In [ ]:
plt.plot(df3['time'], df3['xrsb_flux'], label='GOES XRS-B Flux', color='darkblue')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import seaborn as sns

# Set the style for dark theme with block background
plt.style.use('dark_background')
sns.set_palette("husl")

# Assuming you have your DataFrame as 'df'
# df = pd.read_csv('your_solar_data.csv')  # Load your actual data
# df['time'] = pd.to_datetime(df['time'])  # Convert time to datetime

# Create the plot with block background
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#2d2d44')

# Define colors for different XRSB flux types
colors = {
    'xrsb_flux': '#ffa726',           # Orange
    'xrsb_flux_observed': '#66bb6a',  # Green
    'xrsb_flux_electrons': '#ab47bc'  # Purple
}

# Plot XRSB data (replace with your actual df columns)
# ax.plot(df['time'], df['xrsb_flux'], color=colors['xrsb_flux'],
#         linewidth=2.5, label='XRSB Flux', alpha=0.9)
# ax.plot(df['time'], df['xrsb_flux_observed'], color=colors['xrsb_flux_observed'],
#         linewidth=2, label='XRSB Observed', alpha=0.8)
# ax.plot(df['time'], df['xrsb_flux_electrons'], color=colors['xrsb_flux_electrons'],
#         linewidth=2, label='XRSB Electrons', alpha=0.8)

# Sample data for demonstration (remove when using real data)
# 5-minute intervals for 24 hours

# Plot the data
ax.plot(df3['time'], df3['xrsb_flux'], color=colors['xrsb_flux'], label='XRS-B Flux', alpha=0.9, linewidth=4.5)

plt.yscale('log')
# Customize the plot
ax.grid(True, alpha=0.3, linestyle='--', color='white')
ax.set_ylabel('XRS-B Flux (W/m²)', fontsize=16, color='white', fontweight='bold')
ax.set_xlabel('Time', fontsize=16, color='white', fontweight='bold')
ax.set_title('GOES X-ray Flux - XRS-B Band (1-8 Angstroms)',
             fontsize=17, color='white', fontweight='bold', pad=20)


plt.xlim(pd.to_datetime("2023-07-02"), pd.to_datetime("2023-07-04"))
# # Format legend
legend = ax.legend(loc='upper right', framealpha=0.9, facecolor='#2d2d44',
                  edgecolor='white', fontsize=12)
# legend.get_frame().set_linewidth(1)

# Format ticks
ax.tick_params(axis='both', colors='white', labelsize=11)
ax.tick_params(axis='x', rotation=45)

# Format spines
# for spine in ax.spines.values():
#     spine.set_color('white')
#     spine.set_linewidth(1)



plt.tight_layout()

# Optional: Save the plot
# plt.savefig('xrsb_flux_plot.png', dpi=300, bbox_inches='tight',
#             facecolor='#1a1a2e', edgecolor='none')
plt.savefig('xrsb_flux_plot.png', dpi=500, bbox_inches='tight',
            facecolor='none', edgecolor='none')
plt.show()
